# Классификация на PyTorch Lightning


In [53]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


In [68]:
wine = load_wine()
X = wine.data
y = wine.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Classes: {len(np.unique(y))}")


Train: torch.Size([142, 13]), Test: torch.Size([36, 13])
Classes: 3


In [69]:
class WineDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class WineDataModule(pl.LightningDataModule):
    def __init__(self, X_train, X_test, y_train, y_test, batch_size=16, val_split=0.2):
        # Инициализируем параметры
        super().__init__()
        self.batch_size = batch_size  # размер батча
        self.X_train = X_train  # тензор с тренировочными признаками
        self.X_test = X_test  # тензор с тестовым набором
        self.y_train = y_train  # тензор с таргетом
        self.y_test = y_test  # тестовый таргет
        self.val_split = val_split
    
    def setup(self, stage=None):
        # Готовим датасеты для разных этапов
        if stage == 'fit' or stage is None:
            # Подготовка fit этапа
            full_set = WineDataset(self.X_train, self.y_train)
            train_size = int((1 - self.val_split) * len(full_set))
            val_size = len(full_set) - train_size
            self.wine_train, self.wine_val = random_split(full_set, [train_size, val_size])
            # Подготовка test этапа
        if stage == 'test' or stage is None:
            self.wine_test = WineDataset(self.X_test, self.y_test)
    
    def train_dataloader(self):
        # Загрузка трейна
        return DataLoader(self.wine_train, batch_size=self.batch_size)
    
    def val_dataloader(self):
        # Загрузка валидации
        return DataLoader(self.wine_val, batch_size=self.batch_size)
    
    def test_dataloader(self):
        # Загрузка теста
        return DataLoader(self.wine_test, batch_size=self.batch_size)

In [ ]:
class NeuralNetLightning(pl.LightningModule):
    def __init__(self, input_size, num_classes, lr=0.001):
        # Настраиваем параметры класса
        super().__init__()
        self.save_hyperparameters()  # для хранения lr
        self.l1 = nn.Linear(input_size, 128)
        self.l2 = nn.Linear(128, 64)
        self.l3 = nn.Linear(64, num_classes)
        self.criterion = nn.CrossEntropyLoss()  # поскольку решаем классификацию
    
    def forward(self, x):
        # Делаем прямой проход по трём слоям
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l3(x)
        return x
    
    def training_step(self, batch, batch_idx):
        # Тренируем
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # Проверяем
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True)

    def test_step(self, batch, batch_idx):
        # Тестируем
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)

    def configure_optimizers(self):
        # Оптимизируем
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        return optimizer

In [71]:
data_module = WineDataModule(X_train, X_test, y_train, y_test, batch_size=16)
model = NeuralNetLightning(input_size=13, num_classes=3, lr=0.001)

print(model)


NeuralNetLightning(
  (l1): Linear(in_features=13, out_features=128, bias=True)
  (l2): Linear(in_features=128, out_features=64, bias=True)
  (l3): Linear(in_features=64, out_features=3, bias=True)
  (criterion): CrossEntropyLoss()
)


In [72]:
trainer = pl.Trainer(
    max_epochs=100,
    accelerator='auto',
    devices=1,
    log_every_n_steps=5,
    enable_progress_bar=True
)

trainer.fit(model, data_module)


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | l1        | Linear           | 1.8 K  | train
1 | l2        | Linear           | 8.3 K  | train
2 | l3        | Linear           | 195    | train
3 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
10.2 K    Trainable params
0         Non-trainable params
10.2 K    Total params
0.041     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Artemis\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\Artemis\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [73]:
test_result = trainer.test(model, data_module)
print(f"Test Accuracy: {test_result[0]['test_acc']:.4f}")


c:\Users\Artemis\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                    1.0
        test_loss          0.0026344037614762783
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Test Accuracy: 1.0000


In [74]:
model.eval()
with torch.no_grad():
    logits = model(X_test)
    predicted = torch.argmax(logits, dim=1)
    
from sklearn.metrics import classification_report
print("\nClassification Report:")
print(classification_report(y_test, predicted, target_names=wine.target_names))



Classification Report:
              precision    recall  f1-score   support

     class_0       1.00      1.00      1.00        14
     class_1       1.00      1.00      1.00        14
     class_2       1.00      1.00      1.00         8

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36

